In [1]:
# Importing the libraries
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score , confusion_matrix
_CURRENT_SCRATCH_GRAPH = None

import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
# Importing the dataset
dataset = pd.read_csv('G:\Software\Machine learning\Datasets\Deep learning\Churn_Modelling.csv')

In [3]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.shape

(10000, 14)

In [6]:
#spliting the dataset
x = dataset.iloc[:, 3:12].values
y = dataset.iloc[:, 13].values

In [7]:
x.shape

(10000, 9)

In [8]:
y.shape

(10000,)

## Encoding categorical data

In [9]:
labelencoder_X = LabelEncoder()
x[:, 1] = labelencoder_X.fit_transform(x[:, 1])

In [10]:
x

array([[619, 0, 'Female', ..., 1, 1, 1],
       [608, 2, 'Female', ..., 1, 0, 1],
       [502, 0, 'Female', ..., 3, 1, 0],
       ...,
       [709, 0, 'Female', ..., 1, 0, 1],
       [772, 1, 'Male', ..., 2, 1, 0],
       [792, 0, 'Female', ..., 1, 1, 0]], dtype=object)

In [11]:
labelencoder_X_1 = LabelEncoder()
x[:, 2] = labelencoder_X_1.fit_transform(x[:, 2])

In [12]:
x

array([[619, 0, 0, ..., 1, 1, 1],
       [608, 2, 0, ..., 1, 0, 1],
       [502, 0, 0, ..., 3, 1, 0],
       ...,
       [709, 0, 0, ..., 1, 0, 1],
       [772, 1, 1, ..., 2, 1, 0],
       [792, 0, 0, ..., 1, 1, 0]], dtype=object)

In [13]:
onehotencoder = OneHotEncoder(categories='auto', drop=None, sparse=True, handle_unknown='error')
x = onehotencoder.fit_transform(x).toarray()

In [14]:
x

array([[0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [15]:
x[:,1:]

array([[0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [16]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [17]:
X_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [18]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
X_test

array([[-0.0258285 , -0.01154778, -0.01154778, ...,  0.64365658,
        -0.96668786,  0.96668786],
       [-0.0258285 , -0.01154778, -0.01154778, ...,  0.64365658,
         1.03446007, -1.03446007],
       [-0.0258285 , -0.01154778, -0.01154778, ...,  0.64365658,
        -0.96668786,  0.96668786],
       ...,
       [-0.0258285 , -0.01154778, -0.01154778, ...,  0.64365658,
         1.03446007, -1.03446007],
       [-0.0258285 , -0.01154778, -0.01154778, ...,  0.64365658,
         1.03446007, -1.03446007],
       [-0.0258285 , -0.01154778, -0.01154778, ..., -1.55362351,
         1.03446007, -1.03446007]])

In [20]:
classifier = Sequential()

In [21]:
# Create function for classifier

def build_classifier(optimizer):
    classifier = Sequential()
    
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    classifier.compile(optimizer = optimizer , loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier)

In [23]:
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam' , 'rmsprop']}

In [24]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_